# Create sagemaker endpoints
This notebook creates sagemaker endpoints with the pretrained models

In [ ]:
import boto3
import sagemaker
import os
import numpy as np
import urllib.request

endpoint_instance_type = "ml.m5.xlarge"
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
role = sagemaker.get_execution_role()
print("Your sagemaker s3_bucker is s3://{}".format(s3_bucket))

In [ ]:
models = ["Model-7x7", 
          "Model-11x11", 
          "Model-15x15", 
          "Model-19x19"]

# Get the pretrained models

In [ ]:
if not os.path.exists("pretrained_models"):
    os.mkdir("pretrained_models")

source_url = "https://battlesnake-aws-{region}.s3-{region}.amazonaws.com/pretrainedmodels/{model_name}/Model.tar.gz"
for model in models:
    url = source_url.format(region=sage_session.boto_region_name, model_name=model)
    if not os.path.exists("pretrained_models/{}".format(model)):
        os.mkdir("pretrained_models/{}".format(model))
    urllib.request.urlretrieve(url, "pretrained_models/{}/Model.tar.gz".format(model))

### Make your own S3 bucket and copy the pre-trained models into your own bucket
An s3 bucket is a cloud storage. In this section, we will make a bucket in your AWS account then copy the pre-trained models from public S3 bucket into your own bucket

In [ ]:
s3_client = boto3.client('s3')
for model in models:
    s3_client.upload_file("pretrained_models/{}/Model.tar.gz".format(model), s3_bucket, 
                          "battlesnake-pretrainedmodels/{}/Model.tar.gz".format(model))

# Create a sagemaker endpoint with the bucket


In [ ]:
map_size = (19, 19)
model_data = "s3://{}/battlesnake-pretrainedmodels/Model-{}x{}/Model.tar.gz".format(s3_bucket, map_size[0], map_size[1])
print("Make an endpoint with {}".format(model_data))

In [ ]:
from sagemaker.mxnet import MXNetModel
mxnet_model = MXNetModel(model_data=model_data,
                             entry_point='predict.py',
                             role=role,
                             framework_version='1.6.0',
                             source_dir='battlesnake_inference',
                             name="battlesnake-mxnet123",
                             py_version='py3')
predictor = mxnet_model.deploy(initial_instance_count=1,
                               instance_type=endpoint_instance_type,
                               update_endpoint=False,
                               endpoint_name='battlesnake-endpoint')

# Test the model

In [ ]:
data1 = np.zeros(shape=(1, 2, 3, map_size[0]+2, map_size[1]+2))
data2 = np.zeros(shape=(1, 2))
data3 = np.zeros(shape=(1, 2))
data4 = np.zeros(shape=(1, 2))
action = predictor.predict({"state": data1, "snake_id": data2, 
                   "turn_count": data3, "health": data4})
print("Action to take is {}".format(action))